# Distributed SQL queries with Dask-SQL

In this guide, you'll use [Dask-SQL](https://dask-sql.readthedocs.io/en/latest/), an open-source library which adds a SQL query layer on top of Dask. This allows you to query and transform Dask DataFrames using common SQL operations. You can download {download}`this jupyter notebook <dask-sql.ipynb>` to follow along in your own JupyterLab session.

## Before you start

You'll first need install the necessary packages, For the purposes of this example, we'll do this in a new virtual environment, but you could also install them in whatever environment you're already using for your project.

```bash
$ conda create -n dask-sql-example -c conda-forge python=3.9 dask-sql coiled dask
$ conda activate dask-sql-example
```

You also could use pip, or any other package manager you prefer; conda isn't required.

When you create a cluster, Coiled will automatically replicate your local `dask-sql-example` environment in your cluster.

## Launch your cluster

You'll start by creating a Coiled cluster:

In [ ]:
import coiled
    
cluster = coiled.Cluster(
    name="dask-sql-example",
    n_workers=5,
)

and then connecting Dask to your remote Coiled cluster:

In [2]:
from dask.distributed import Client

# connect to the remote cluster
client = Client(cluster)

## Getting started with Dask-SQL

The main interface for interacting with Dask-SQL is the `dask_sql.Context` object. It allows you to register Dask DataFrames as data sources and can convert SQL queries to Dask DataFrame operations.

In [4]:
from dask_sql import Context

c = Context()

You'll use the `dask.datasets.timeseries` dataset of random timeseries data:

In [5]:
import dask

# Load dataset
df = dask.datasets.timeseries(
    "2000", "2005", partition_freq="2w"
)

You can then use the `dask_sql.Context` to assign a table name to this Dask DataFrame, and use that table name for SQL queries:

In [6]:
# Register the Dask DataFrame df as a table
# use persist=True for reducing overhead for multiple queries
c.register_dask_table(df, "timeseries", persist=True)

# Perform a SQL operation on the "timeseries" table
result = c.sql("SELECT count(1) FROM timeseries")

Note that this returned another Dask DataFrame and no computation has been run yet. This is similar to other Dask operations, which are lazily evaluated. We can use `compute` to run the computation on our cluster.

In [7]:
result.compute()

,COUNT(1)
0,157248000


You've run your first SQL query with Dask-SQL! Let’s try out some more complex queries.

## More complex SQL examples

With Dask-SQL we can run more complex SQL statements like, for example, a groupby-aggregation:

In [8]:
c.sql('SELECT max(y) as "max", name FROM timeseries GROUP BY name').compute()

,max,name
0,1.000000,Alice
1,1.000000,Bob
2,1.000000,Charlie
3,0.999999,Dan
4,1.000000,Edith
5,1.000000,Frank
6,1.000000,George
7,1.000000,Hannah
8,1.000000,Ingrid
9,0.999999,Jerry


The equivalent operation using the Dask DataFrame API would be `df.groupby("passenger_count").tip_amount.mean().compute()`. We can build up complexity by adding a ``WHERE`` clause to filter for certain values of ``x``:

In [9]:
c.sql("""
    SELECT name, AVG(y) as "average"
    FROM timeseries 
    WHERE x > 0.2
    GROUP BY name
""").compute()

,name,average
0,Alice,-4.057128e-04
1,Bob,3.807399e-04
2,Charlie,1.664932e-04
3,Dan,2.467381e-04
4,Edith,9.742376e-05
5,Frank,-9.143104e-05
6,George,-2.146390e-04
7,Hannah,-6.085306e-05
8,Ingrid,2.383526e-05
9,Jerry,-2.286118e-04


Once you're done, you can shutdown the cluster (it will shutdown automatically after 20 minutes of inactivity):

In [10]:
cluster.close()
client.close()

## Next Steps

For a more in-depth look at what you can do with Dask-SQL, see the [Operating on Dask DataFrames with SQL](https://examples.dask.org/sql.html) how-to guide. You can also reference the [Dask-SQL docs](https://dask-sql.readthedocs.io/) or [GitHub repo](https://github.com/dask-contrib/dask-sql).